In [ ]:
from google.oauth2 import service_account
from googleapiclient.discovery import build
import io
from googleapiclient.http import MediaIoBaseDownload
import os

# Path to your service account credentials JSON
SERVICE_ACCOUNT_FILE =  ''                       # 'C:/Users/api_auth_file.json' for example

# Folder ID of your Drive folder containing thermal images
FOLDER_ID = '1cTzfg8extLH7V3hiXgFXgmVvUkCz4jkc'  # Dumbach ID for example

# Scopes for read-only access to Drive files
SCOPES = ['https://www.googleapis.com/auth/drive.readonly']

def authenticate_drive():
    """
    Authenticate to Google Drive using a service account.
    """
    creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES
    )
    service = build('drive', 'v3', credentials=creds)
    return service

def list_files_in_folder(service, folder_id):
    """
    List all files in the specified Drive folder.
    Returns a list of dicts: [{'id': ..., 'name': ..., 'mimeType': ...}, ...]
    """
    results = []
    page_token = None
    query = f"'{folder_id}' in parents and trashed = false"
    while True:
        response = service.files().list(
            q=query,
            spaces='drive',
            fields='nextPageToken, files(id, name, mimeType)',
            pageToken=page_token
        ).execute()
        results.extend(response.get('files', []))
        page_token = response.get('nextPageToken', None)
        if not page_token:
            break
    return results

def download_file(service, file_id, destination_path):
    """
    Download a file from Drive by file ID.
    """
    request = service.files().get_media(fileId=file_id)
    os.makedirs(os.path.dirname(destination_path), exist_ok=True)
    fh = io.FileIO(destination_path, 'wb')
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while not done:
        status, done = downloader.next_chunk()
        print(f"Downloading {os.path.basename(destination_path)}: {int(status.progress() * 100)}%")
    fh.close()

if __name__ == "__main__":
    # 1. Authenticate
    drive_service = authenticate_drive()

    # 2. List files in the folder
    files = list_files_in_folder(drive_service, FOLDER_ID)
    print(f"Found {len(files)} files in folder.")

    # 3. Download each image file
    for file in files:
        file_id = file['id']
        name = file['name']
        mime = file['mimeType']
        print(f"Processing file: {name} (ID: {file_id}, Type: {mime})")
        # Only download common image types
        if mime.startswith('image/'):
            destination = os.path.join('downloaded_images', name)
            download_file(drive_service, file_id, destination)
        else:
            print(f"Skipping non-image file: {name}")


Found 35 files in folder.
Processing file: CH12.jpeg (ID: 1md_-NGQi8bLifzeOr_ooEm2AYtkji63h, Type: image/jpeg)
Processing file: CH10.jpeg (ID: 1SAegw0rUga4-ttaj6yOVORBt5lU7X2u_, Type: image/jpeg)
Processing file: CH08.jpeg (ID: 1ciEy6orP7HUPDSNOmj3FTeNds6HFg_3_, Type: image/jpeg)
Processing file: CH05.jpeg (ID: 1y8-CvTFevhZY2Cpr8SwcCvoSdsbk6Bx_, Type: image/jpeg)
Processing file: CH04.jpeg (ID: 16FGbzrXXPGw82Lm2ACJN9SXxFF3g6ooz, Type: image/jpeg)
Processing file: CH03.jpeg (ID: 1BN5pnV8gtyOQ3WbMYJiY0O5uDgYE9A69, Type: image/jpeg)
Processing file: CH02.jpeg (ID: 1YD6udk_dqtGNKU7g5LjT8OPHx1qTB86r, Type: image/jpeg)
Processing file: CH01.jpeg (ID: 1Bwj4X-qDX4t4jiVSJHYsITZIUPARU4l5, Type: image/jpeg)
Processing file: CH09.jpeg (ID: 1CI4wMntPjRIqToLcSsDZ796YphrkxHx-, Type: image/jpeg)
Processing file: CH07.jpeg (ID: 1deeWnsHKtfbqRKrPZgRmVLG44HN704WC, Type: image/jpeg)
Processing file: CH06.jpeg (ID: 15CPE-Dgbkzb9POVyD18MAocyXPzqeHmf, Type: image/jpeg)
Processing file: CH13.jpeg (ID: 1e1Jdgs